# 1. Introduction about dataset
## Context
* The **SMS Spam Collection** is a set of SMS tagged messages that have been collected for SMS Spam research.
* It contains one set of SMS messages in English of **5,574** messages, tagged acording being **ham** (legitimate) or **spam.**

## Content
* The files contain one message per line.
* Each line is composed by two columns:
 * v1 contains the label (ham or spam) and
 * v2 contains the raw text.




**Thanks to:-**
* [SMS Spam Collection](https://archive.ics.uci.edu/dataset/228/sms+spam+collection)

# 2. Import libraries
- To building a spam classifier we need to perform text preprocessing.
- For this required necessary libraries and tools to import for
  - data manipulation,
  - text cleaning,
  - stemming,
  - lemmatization, and
  - removal of common stopwords.
  These steps help prepare raw text data for machine learning models, improving their accuracy and performance.

In [ ]:
# provides tools for text preprocessing, tokenization, stemming, lemmatization, stopwords removal, and more.
import nltk

# Helps in loading, analyzing, and manipulating the dataset efficiently. It is typically used to read the data, handle missing values, and structure the data in a tabular format, particularly when working with structured data like CSV files.
import pandas as pd

# for cleaning text data, such as removing special characters, punctuations, and other unwanted patterns from the text.
import re

# simplify words in text data to their base form, which helps in reducing the dimensionality of text data and improving the performance of the model.
from nltk.stem import PorterStemmer

# convert words to their base or dictionary form (e.g., "better" to "good"). It is more context-aware than stemming, making it more effective in NLP tasks.
from nltk.stem import WordNetLemmatizer

# removing the frequently occurring words (e.g., "the", "and", "is") that do not contribute significantly to the classification task, thereby improving model efficiency and accuracy.
from nltk.corpus import stopwords

#### Download the nltk library's data

In [ ]:
nltk.download('all')

#### Calling others dependent libraries

In [ ]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

#### copying the raw spam data from **github**

In [ ]:
url_new="https://raw.githubusercontent.com/akdubey2k/NLP/main/Spam_Classifier/SpamCollection.csv"
url_org='https://raw.githubusercontent.com/akdubey2k/NLP/main/Spam_Classifier/SMSSpamCollection.csv'

#### Read data from **github** repositary using **pandas**.

In [ ]:
df=pd.read_csv(url_new, sep='\t')
df.head()

,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
0,ham,Ok lar... Joking wif u oni...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,U dun say so early hor... U c already then say...
3,ham,"Nah I don't think he goes to usf, he lives aro..."
4,spam,FreeMsg Hey there darling it's been 3 week's n...


#### Creating a **pandas** dataframe by inserting new columns named as **label**, **message** and **index** by default true.

In [ ]:
df=pd.read_csv(url_new, sep='\t', names=['label', 'message'])
df.head()

In [ ]:
df=pd.read_csv(url_org, sep='\t', names=['label', 'message'])
df.head()

#### Creation of **Stemming** and **Lemmatization** object

In [ ]:
ps=PorterStemmer()
wnl=WordNetLemmatizer()

#### Data **Cleaning** and **preprocessing**

In [ ]:
corpus=[]
for i in range(0, len(df)):
  data=re.sub('[^a-zA-Z]', ' ', df['message'][i])
  # print(data)
  data=data.lower()
  data=data.split()
  #data=[ps.stem(word) for word in data if word not in set(stopwords.words('english'))]
  data=[wnl.lemmatize(word) for word in data if word not in set(stopwords.words('english'))]
  data=' '.join(data)
  corpus.append(data)

# print(corpus)

#### **Bag of word** creation for spam classification

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=2500)
X=cv.fit_transform(corpus).toarray()
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

#### get **dummies** from **pandas** dataframe for the **output/dependent** variable (**label** column)

In [ ]:
y=pd.get_dummies(df['label'])
y

#### As there is only two dummy varibales, so we do not need two column... we can convert this **dummy variable trap** by using single column.

In [ ]:
y=y.iloc[:,1].values # all rows and only one column.... values need to be present
y

array([0, 0, 1, ..., 0, 0, 0], dtype=uint8)

#### Split the data into **training** and **test** set, using **sklearn**
X (fit_transform(corpus)) => independent variable and

y (single dummy varible) => depedent variable

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.20, random_state=0)

#### Classification (Training) model using **Naive Bayes Classifier**


In [ ]:
from sklearn.naive_bayes import MultinomialNB
mtnb=MultinomialNB()
mtnb.fit(X, y)

MultinomialNB()

In [ ]:
mtnb=mtnb.fit(X_train, y_train)
y_pred=mtnb.predict(X_test)
y_pred

array([0, 1, 0, ..., 0, 1, 0], dtype=uint8)

#### Classification between **y_test** and **y_pred** values using **Confusion Matrix**

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred) # 2 x 2 matrix
cm

array([[946,   9],
       [ 10, 150]])

#### Check the accuracy score of model

In [ ]:
from sklearn.metrics import accuracy_score
ac=accuracy_score(y_test, y_pred)
ac

0.9829596412556054